In [1]:
import affine, concurrent, cStringIO, glob, IPython, json, os, PIL, sys, thread, time, traceback


In [2]:
class SimpleProcessPoolExecutor(concurrent.futures.ProcessPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleProcessPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleProcessPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleProcessPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleProcessPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleProcessPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleProcessPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

class Stopwatch:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, type, value, traceback):
        sys.stdout.write('%s took %.1f seconds\n' % (self.name, time.time() - self.start))


def subdivide(tname):
    c = "CREATE TABLE IF NOT EXISTS %s_subdivided AS SELECT ST_SubDivide(geom) AS geom, gid as %s_gid FROM %s;" % (tname, tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd
    
def create_index(tname):
    c = "CREATE INDEX IF NOT EXISTS %s_geom_idx ON %s USING GIST (geom);" % (tname, tname)
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    
    
def vacuum_table(tname):
    c = "VACUUM ANALYZE %s;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd   
    
def make_valid(tname):
    c = "UPDATE %s SET geom=ST_MakeValid(geom) WHERE ST_IsValid(geom) = false;" % tname
    cmd = 'psql -d biodiversity -c "%s"' % c 
    !$cmd    

In [3]:
make_valid('wdpa_subdivided')


NOTICE:  Interior is disconnected at or near point 145.4310042300001 -38.485951130999922
NOTICE:  Ring Self-intersection at or near point 113.71423326800004 -22.496189295999955
NOTICE:  Ring Self-intersection at or near point -63.006954359999952 44.828263341000081
NOTICE:  Interior is disconnected at or near point -104.47594568499994 51.779782855000065
NOTICE:  Ring Self-intersection at or near point -106.66735300999994 51.086668690000067
NOTICE:  Interior is disconnected at or near point -63.107560708999927 -38.568429071999901
NOTICE:  Interior is disconnected at or near point -67.798440867999943 -54.402530205999938
NOTICE:  Interior is disconnected at or near point -67.79471641899994 -54.410773763999941
NOTICE:  Interior is disconnected at or near point -67.766213295999933 -54.41473009799995
NOTICE:  Interior is disconnected at or near point -67.762555310999915 -54.412354501999914
NOTICE:  Ring Self-intersection at or near point -67.737631085999908 -54.422000496999942
NOTICE:  Interi

NOTICE:  Ring Self-intersection at or near point 10.381861614000059 57.333666710000045
NOTICE:  Ring Self-intersection at or near point 10.273764606000043 57.322502198000052
NOTICE:  Ring Self-intersection at or near point 10.268266105000066 57.29741462000004
NOTICE:  Ring Self-intersection at or near point 10.289904402000047 57.287196483000059
NOTICE:  Ring Self-intersection at or near point 10.447449804000087 57.424457117000031
NOTICE:  Ring Self-intersection at or near point 10.387912431000075 57.508427187000052
NOTICE:  Ring Self-intersection at or near point 10.407619578000038 57.499009249000039
NOTICE:  Ring Self-intersection at or near point 10.400131673000089 57.515779852000037
NOTICE:  Ring Self-intersection at or near point 10.448542657000075 57.657770787000061
NOTICE:  Interior is disconnected at or near point 10.453342418000091 57.663873516000081
NOTICE:  Interior is disconnected at or near point 10.605112686000041 57.750583371000062
NOTICE:  Ring Self-intersection at or ne

NOTICE:  Interior is disconnected at or near point 8.652557922000085 56.304652392000037
NOTICE:  Ring Self-intersection at or near point 8.6574777030000405 56.309336545000065
NOTICE:  Ring Self-intersection at or near point 8.4927702090000707 56.319427773000086
NOTICE:  Ring Self-intersection at or near point 8.909592845000077 56.444581029000062
NOTICE:  Interior is disconnected at or near point 8.5343494530000612 56.365969902000074
NOTICE:  Ring Self-intersection at or near point 8.1240902470000833 56.326210562000085
NOTICE:  Ring Self-intersection at or near point 145.39712462500006 -15.955603324999913
NOTICE:  Ring Self-intersection at or near point 11.073349191000091 57.301648711000041
NOTICE:  Ring Self-intersection at or near point 11.072557176000089 57.302463616000068
NOTICE:  Ring Self-intersection at or near point 10.958526676000076 57.228572889000077
NOTICE:  Ring Self-intersection at or near point 10.911129495000068 57.244027230000086
NOTICE:  Ring Self-intersection at or ne

NOTICE:  Interior is disconnected at or near point 3.5688646180000774 51.162976075000074
NOTICE:  Interior is disconnected at or near point 3.4881605850000597 50.758715973000051
NOTICE:  Interior is disconnected at or near point 4.3195126850000634 51.157479379000051
NOTICE:  Ring Self-intersection at or near point 4.123160513000073 50.86668790300007
NOTICE:  Interior is disconnected at or near point 4.3438126510000643 50.371462813000051
NOTICE:  Ring Self-intersection at or near point -80.536980284999913 27.968629892000081
NOTICE:  Ring Self-intersection at or near point -82.508916996999915 27.21233295400009
NOTICE:  Ring Self-intersection at or near point 23.413723276000042 43.214043487000083
NOTICE:  Ring Self-intersection at or near point 22.559244161000038 42.163693596000087
NOTICE:  Ring Self-intersection at or near point 23.940980922000051 41.719414913000037
NOTICE:  Ring Self-intersection at or near point 23.405653860000086 41.514675453000052
NOTICE:  Interior is disconnected at

NOTICE:  Ring Self-intersection at or near point 8.896612358000084 52.000936239000055
NOTICE:  Ring Self-intersection at or near point 7.8830044340000427 50.932588887000065
NOTICE:  Ring Self-intersection at or near point 8.1988084760000675 51.293124090000049
NOTICE:  Ring Self-intersection at or near point 9.1265591990000416 52.107599502000085
NOTICE:  Ring Self-intersection at or near point 7.6763111040000922 50.914200390000076
NOTICE:  Ring Self-intersection at or near point 8.5688841170000387 51.441578053000057
NOTICE:  Ring Self-intersection at or near point 7.0135576080000419 51.17684812400006
NOTICE:  Ring Self-intersection at or near point 6.1739034460000539 50.742907572000036
NOTICE:  Ring Self-intersection at or near point 8.3096536740000602 50.95803268800006
NOTICE:  Ring Self-intersection at or near point 7.3951071200000911 51.179047003000051
NOTICE:  Ring Self-intersection at or near point 7.7432532070000661 51.062375343000042
NOTICE:  Ring Self-intersection at or near poi

...

NOTICE:  Ring Self-intersection at or near point -85.441092906999927 31.964175139000076
NOTICE:  Ring Self-intersection at or near point -91.632841628999927 34.305177527000069
NOTICE:  Ring Self-intersection at or near point -96.57984358199991 48.889989896000088
NOTICE:  Interior is disconnected at or near point -87.429792248999945 30.641402954000057
NOTICE:  Ring Self-intersection at or near point -105.96891387599993 33.64923809600009
NOTICE:  Ring Self-intersection at or near point -108.95779630499993 31.779359020000072
NOTICE:  Ring Self-intersection at or near point -109.28410223399993 31.880277835000072
NOTICE:  Ring Self-intersection at or near point -108.6979129209999 39.678786051000088
NOTICE:  Ring Self-intersection at or near point -110.2352128849999 38.23849944400007
NOTICE:  Ring Self-intersection at or near point -117.72038199999992 35.236241000000064
NOTICE:  Ring Self-intersection at or near point -89.316049316999909 35.023510314000077
NOTICE:  Ring Self-intersection at 

NOTICE:  Ring Self-intersection at or near point -89.126368420999938 46.282563860000039
NOTICE:  Ring Self-intersection at or near point -89.15784974099995 46.43414609000007
NOTICE:  Ring Self-intersection at or near point -88.949372929999925 46.188621370000078
NOTICE:  Ring Self-intersection at or near point -88.949954689999913 46.434051910000051
NOTICE:  Ring Self-intersection at or near point -88.866238088999921 46.340295170000047
NOTICE:  Ring Self-intersection at or near point -88.762316014999897 46.231680650000044
NOTICE:  Ring Self-intersection at or near point -88.740987699999948 46.36219514000004
NOTICE:  Ring Self-intersection at or near point -89.012156839999932 46.44862696000007
NOTICE:  Interior is disconnected at or near point -88.804195509999943 46.448687800000073
NOTICE:  Ring Self-intersection at or near point -88.721428038999932 46.477814180000053
NOTICE:  Ring Self-intersection at or near point -88.730822866999915 46.661001310000074
NOTICE:  Ring Self-intersection at

NOTICE:  Interior is disconnected at or near point -89.474998207999931 30.202877841000088
NOTICE:  Ring Self-intersection at or near point -89.458027526999899 30.192114618000062
NOTICE:  Interior is disconnected at or near point -89.461688205999906 30.205063861000042
NOTICE:  Ring Self-intersection at or near point -89.432091231999948 30.395296224000049
NOTICE:  Interior is disconnected at or near point -89.389227030999905 30.355358641000066
NOTICE:  Interior is disconnected at or near point -89.266089758999897 30.366561007000087
NOTICE:  Ring Self-intersection at or near point -118.06159111199992 44.085696974000086
NOTICE:  Ring Self-intersection at or near point -112.81540485599993 44.710930832000088
NOTICE:  Ring Self-intersection at or near point -118.31761580099993 42.092892691000088
NOTICE:  Ring Self-intersection at or near point -118.27502632099993 42.063672302000043
NOTICE:  Ring Self-intersection at or near point -118.3175014709999 42.111065552000071
NOTICE:  Ring Self-inters

NOTICE:  Ring Self-intersection at or near point -71.28254036699991 41.763804117000063
NOTICE:  Interior is disconnected at or near point -70.82006167399993 43.197263934000034
NOTICE:  Ring Self-intersection at or near point -81.603208976999952 25.890842957000075
NOTICE:  Ring Self-intersection at or near point -81.624703808999925 25.899379787000044
NOTICE:  Ring Self-intersection at or near point -81.547840651999934 25.850397700000087
NOTICE:  Ring Self-intersection at or near point -166.5367503569999 53.899741625000047
NOTICE:  Ring Self-intersection at or near point -166.99924235199992 54.305409936000046
NOTICE:  Ring Self-intersection at or near point -75.931615991999934 36.663050611000074
NOTICE:  Ring Self-intersection at or near point -71.154037089999917 44.817337845000054
NOTICE:  Ring Self-intersection at or near point -89.025953446999949 30.407363702000055
NOTICE:  Ring Self-intersection at or near point -89.003129718999901 30.424870372000044
NOTICE:  Ring Self-intersection a

NOTICE:  Ring Self-intersection at or near point 14.60944519700007 45.049479154000039
NOTICE:  Ring Self-intersection at or near point 14.970788536000043 45.446106437000083
NOTICE:  Ring Self-intersection at or near point 14.825912306000077 45.422841968000057
NOTICE:  Ring Self-intersection at or near point 14.474023134000049 50.470913405000033
NOTICE:  Ring Self-intersection at or near point 23.745775746000049 63.997519914000065
NOTICE:  Ring Self-intersection at or near point 26.289212430000077 42.376139497000054
NOTICE:  Ring Self-intersection at or near point 20.922121751000077 48.004478278000079
NOTICE:  Ring Self-intersection at or near point 30.088231903000064 61.86267838100008
NOTICE:  Ring Self-intersection at or near point 18.79312647200004 47.786110576000056
NOTICE:  Ring Self-intersection at or near point 26.12350273900006 55.381753113000059
NOTICE:  Ring Self-intersection at or near point 13.385295657000086 50.476620050000065
NOTICE:  Ring Self-intersection at or near poin

In [4]:
create_index('wdpa_subdivided')


CREATE INDEX


In [5]:
vacuum_table('wdpa_subdivided')


VACUUM
